In [14]:
import fastf1 as f1
import pandas as pd
import numpy as np
import datetime

In [ ]:
f1.Cache.enable_cache('cache\\')
f1.Cache.offline_mode(enabled=True)
seasons = [2023,2024]
drop_labels=['Time', 'Stint','PitOutTime', 'PitInTime', 'Sector1Time', 'Sector2Time', 'Sector3Time',
                'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
                'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'IsPersonalBest',
                'FreshTyre', 'Team', 'LapStartTime',
                'LapStartDate', 'TrackStatus', 'Position', 'Deleted', 'DeletedReason',
                'FastF1Generated', 'IsAccurate']

In [16]:
def update_cache(_year:int):
    e1 = pd.DataFrame(data=f1.get_event_schedule(_year))
    e1 = e1[e1['RoundNumber']!=0]
    for es in e1['EventName']:
        e2 = f1.get_session(_year,gp=es,identifier='R')
        e2.load()

In [27]:
def update_user_data(_season:int):
    df=pd.DataFrame()
    for season in _season:
        e1 = pd.DataFrame(data=f1.get_event_schedule(season))
        e1 = e1[e1['RoundNumber']!=0]
        for es in e1['EventName']:
            e2 = f1.get_session(season,es,'R')
            if (e2.date.date()<datetime.date.today()):
                e2.load()
                temp=e2.laps.drop(labels=drop_labels,axis=1)
                temp['Year'] = e2.event.EventDate.year
                temp['GrandPrix'] = e2.event.EventName
                temp['LapTime'] = e2.laps['LapTime'].dt.total_seconds()
                temp = pd.concat([temp,e2.laps.get_weather_data().drop(labels=['Time','Rainfall'],axis=1).reset_index(drop=True)]
                                 ,axis=1)
                df = pd.concat([df,temp])
    df.to_csv('lap_data\\'+str(_season)+'.csv',index=None)            

In [ ]:
update_user_data([2022])